# Modelamiento Predictivo

Dataset: 1.14M registros

- Sampling estratificado para reducir tiempo de entrenamiento
- Hiperparámetros optimizados para datasets grandes
- LightGBM (más rápido que XGBoost)
- Métricas de evaluación completas
- Validación cruzada opcional

In [2]:
import time
import pandas as pd
import numpy as np
import joblib
import os

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import xgboost as xgb
from lightgbm import LGBMClassifier

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

RANDOM_STATE = 42

## 1. Carga y Preparación de Datos

In [3]:
# Carga de datos
ruta_datos = '../data/procesada/master_dataset.csv'
df = pd.read_csv(ruta_datos)

print(f"Dataset completo: {df.shape}")

# Conversión de fecha
df['Fecha_consulta'] = pd.to_datetime(df['Fecha_consulta'])

# Feature Engineering
df['Hora_Consulta'] = df['Fecha_consulta'].dt.hour
df['Dia_Semana'] = df['Fecha_consulta'].dt.dayofweek
df['Es_FinDeSemana'] = (df['Dia_Semana'] >= 5).astype(int)
df['Ratio_Deuda_Edad'] = df['Monto_adeudado'] / (df['Edad'] + 0.1)
df['Ratio_Duracion_Espera'] = df['Duracion_llamada'] / (df['Tiempo_en_espera'] + 1)

# Separación X e y
cols_drop = ['ID_Cuenta', 'Fecha_consulta', 'Correo', 'y']
X = df.drop(columns=cols_drop, errors='ignore')
y = df['y']

print(f"\nDimensiones finales: X={X.shape}, y={y.shape}")
print(f"\nDistribución del target:")
print(y.value_counts(normalize=True))

Dataset completo: (1140532, 24)

Dimensiones finales: X=(1140532, 25), y=(1140532,)

Distribución del target:
y
0    0.749257
1    0.250743
Name: proportion, dtype: float64


## 2. Configuración de Preprocesamiento

In [4]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

# Identificación de tipos de columnas
cols_numericas = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cols_categoricas = X.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\nColumnas numéricas: {len(cols_numericas)}")
print(f"Columnas categóricas: {len(cols_categoricas)}")

# Pipeline de preprocesamiento
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, cols_numericas),
        ('cat', categorical_transformer, cols_categoricas)
    ]
)

print("\nPipeline configurado")

Train: (912425, 25), Test: (228107, 25)

Columnas numéricas: 7
Columnas categóricas: 16

Pipeline configurado


## 3. Definición de Modelos

Modelos optimizados para datasets grandes (1M+ registros):
- Logistic Regression: solver='saga' (más rápido para datos grandes)
- Random Forest: n_estimators reducido, max_depth limitado
- XGBoost: tree_method='hist' (más rápido)
- LightGBM: el más rápido para datos grandes

In [5]:
# Ratio de balanceo para XGBoost y LightGBM
ratio_balanceo = float(np.sum(y_train == 0)) / np.sum(y_train == 1)
print(f"Ratio de balanceo: {ratio_balanceo:.2f}")

# Diccionario de modelos
pipelines_modelos = {
    "Logistic Regression": Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(
            random_state=RANDOM_STATE,
            max_iter=1000,
            solver='saga',  # Más rápido para datasets grandes
            class_weight='balanced',
            n_jobs=-1
        ))
    ]),
    
    "Random Forest": Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(
            n_estimators=100,  # Reducido de 200 para velocidad
            max_depth=15,  # Limitar profundidad
            min_samples_split=100,  # Evitar overfitting
            min_samples_leaf=50,
            random_state=RANDOM_STATE,
            n_jobs=-1,
            class_weight='balanced',
            verbose=0
        ))
    ]),
    
    "XGBoost": Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', xgb.XGBClassifier(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            tree_method='hist',  # Mucho más rápido para datos grandes
            scale_pos_weight=ratio_balanceo,
            random_state=RANDOM_STATE,
            n_jobs=-1,
            eval_metric='logloss'
        ))
    ]),
    
    "LightGBM": Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LGBMClassifier(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            num_leaves=31,
            scale_pos_weight=ratio_balanceo,
            random_state=RANDOM_STATE,
            n_jobs=-1,
            verbose=-1
        ))
    ])
}

print(f"\nModelos configurados: {list(pipelines_modelos.keys())}")

Ratio de balanceo: 2.99

Modelos configurados: ['Logistic Regression', 'Random Forest', 'XGBoost', 'LightGBM']


## 4. Entrenamiento y Evaluación

Se entrenan todos los modelos y se registran las métricas de desempeño.

In [6]:
resultados = []
modelos_entrenados = {}

print("Iniciando entrenamiento...\n")

for nombre, pipeline in pipelines_modelos.items():
    print(f"Entrenando {nombre}...")
    
    # Entrenamiento con medición de tiempo
    start_time = time.time()
    pipeline.fit(X_train, y_train)
    tiempo_entrenamiento = time.time() - start_time
    
    # Guardar modelo
    modelos_entrenados[nombre] = pipeline
    
    # Predicciones
    y_pred = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)[:, 1]
    
    # Métricas
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    auc = roc_auc_score(y_test, y_prob)
    f1 = f1_score(y_test, y_pred)
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    
    resultados.append({
        "Modelo": nombre,
        "Accuracy": report['accuracy'],
        "Precision": report['1']['precision'],
        "Recall": report['1']['recall'],
        "F1-Score": f1,
        "ROC-AUC": auc,
        "Tiempo (s)": round(tiempo_entrenamiento, 2)
    })
    
    print(f"  Completado en {tiempo_entrenamiento:.2f}s | ROC-AUC: {auc:.4f}")
    print(f"  Matriz de confusión: TN={cm[0,0]}, FP={cm[0,1]}, FN={cm[1,0]}, TP={cm[1,1]}\n")

print("Entrenamiento completado")

Iniciando entrenamiento...

Entrenando Logistic Regression...


/Users/feliperuizzea/miniforge3/envs/ds_repaso/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


  Completado en 349.05s | ROC-AUC: 0.5137
  Matriz de confusión: TN=81981, FP=88930, FN=26410, TP=30786

Entrenando Random Forest...
  Completado en 48.22s | ROC-AUC: 0.7420
  Matriz de confusión: TN=114793, FP=56118, FN=18761, TP=38435

Entrenando XGBoost...
  Completado en 3.30s | ROC-AUC: 0.6222
  Matriz de confusión: TN=92944, FP=77967, FN=21967, TP=35229

Entrenando LightGBM...


/Users/feliperuizzea/miniforge3/envs/ds_repaso/lib/python3.11/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


  Completado en 3.17s | ROC-AUC: 0.6065
  Matriz de confusión: TN=90602, FP=80309, FN=22287, TP=34909

Entrenamiento completado


/Users/feliperuizzea/miniforge3/envs/ds_repaso/lib/python3.11/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


## 5. Comparación de Resultados

In [7]:
# DataFrame de resultados
df_resultados = pd.DataFrame(resultados).sort_values(by='ROC-AUC', ascending=False)

print("Tabla Comparativa de Modelos:\n")
print(df_resultados.to_string(index=False))

# Selección del mejor modelo
mejor_modelo_nombre = df_resultados.iloc[0]['Modelo']
mejor_pipeline = modelos_entrenados[mejor_modelo_nombre]
mejor_auc = df_resultados.iloc[0]['ROC-AUC']

print(f"\n{'='*60}")
print(f"Modelo seleccionado: {mejor_modelo_nombre}")
print(f"ROC-AUC: {mejor_auc:.4f}")
print(f"{'='*60}")

Tabla Comparativa de Modelos:

             Modelo  Accuracy  Precision   Recall  F1-Score  ROC-AUC  Tiempo (s)
      Random Forest  0.671737   0.406492 0.671988  0.506560 0.741951       48.22
            XGBoost  0.561899   0.311221 0.615935  0.413505 0.622189        3.30
           LightGBM  0.550229   0.302982 0.610340  0.404944 0.606473        3.17
Logistic Regression  0.494360   0.257159 0.538254  0.348037 0.513717      349.05

Modelo seleccionado: Random Forest
ROC-AUC: 0.7420


## 6. Reporte Detallado del Mejor Modelo

In [8]:
# Predicciones del mejor modelo
y_pred_best = mejor_pipeline.predict(X_test)
y_prob_best = mejor_pipeline.predict_proba(X_test)[:, 1]

# Reporte completo
print(f"Classification Report - {mejor_modelo_nombre}:\n")
print(classification_report(y_test, y_pred_best, target_names=['Clase 0', 'Clase 1']))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_best)
print(f"\nMatriz de Confusión:")
print(f"                 Predicho 0    Predicho 1")
print(f"Real 0 (TN/FP)   {cm[0,0]:>10}    {cm[0,1]:>10}")
print(f"Real 1 (FN/TP)   {cm[1,0]:>10}    {cm[1,1]:>10}")

Classification Report - Random Forest:

              precision    recall  f1-score   support

     Clase 0       0.86      0.67      0.75    170911
     Clase 1       0.41      0.67      0.51     57196

    accuracy                           0.67    228107
   macro avg       0.63      0.67      0.63    228107
weighted avg       0.75      0.67      0.69    228107


Matriz de Confusión:
                 Predicho 0    Predicho 1
Real 0 (TN/FP)       114793         56118
Real 1 (FN/TP)        18761         38435


## 7. Serialización del Mejor Modelo

In [9]:
# Crear directorio si no existe
os.makedirs('../models', exist_ok=True)

# Guardar modelo
model_path = f'../models/best_model_{mejor_modelo_nombre.replace(" ", "_").lower()}.joblib'
joblib.dump(mejor_pipeline, model_path)

print(f"Modelo guardado en: {model_path}")

# Guardar también los resultados
df_resultados.to_csv('../models/model_comparison.csv', index=False)
print(f"Tabla de comparación guardada en: ../models/model_comparison.csv")

Modelo guardado en: ../models/best_model_random_forest.joblib
Tabla de comparación guardada en: ../models/model_comparison.csv


## 8. Análisis de Importancia de Variables (opcional)

Solo funciona para modelos tree-based (Random Forest, XGBoost, LightGBM)

In [10]:
if mejor_modelo_nombre in ["Random Forest", "XGBoost", "LightGBM"]:
    # Obtener el clasificador del pipeline
    classifier = mejor_pipeline.named_steps['classifier']
    
    # Obtener nombres de features después del preprocesamiento
    preprocessor_fit = mejor_pipeline.named_steps['preprocessor']
    
    # Nombres de features numéricas
    num_features = cols_numericas
    
    # Nombres de features categóricas (después de OneHotEncoding)
    if len(cols_categoricas) > 0:
        cat_encoder = preprocessor_fit.named_transformers_['cat'].named_steps['onehot']
        cat_features = cat_encoder.get_feature_names_out(cols_categoricas).tolist()
    else:
        cat_features = []
    
    all_features = num_features + cat_features
    
    # Importancias
    importances = classifier.feature_importances_
    
    # DataFrame de importancias
    feature_importance_df = pd.DataFrame({
        'Feature': all_features,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)
    
    print(f"\nTop 20 features más importantes:\n")
    print(feature_importance_df.head(20).to_string(index=False))
    
    # Guardar importancias
    feature_importance_df.to_csv('../models/feature_importances.csv', index=False)
    print(f"\nImportancias guardadas en: ../models/feature_importances.csv")
else:
    print(f"\nEl modelo {mejor_modelo_nombre} no soporta feature importances")


Top 20 features más importantes:

                       Feature  Importance
              Tiempo_en_espera    0.090964
              Ratio_Deuda_Edad    0.088334
                Monto_adeudado    0.087864
         Ratio_Duracion_Espera    0.086787
              Duracion_llamada    0.085841
                          Edad    0.066782
                    usa_app_no    0.011344
             Antiguedad_Legend    0.011314
             Forma_pago_online    0.011052
                    usa_app_si    0.010768
          Tipo_persona_soltero    0.010559
           Recomienda_marca_si    0.010471
              Ha_caido_mora_no    0.010318
           Recomienda_marca_no    0.010011
              Ha_caido_mora_si    0.009606
      Forma_pago_No_Registrado    0.009194
      Tipo_persona_unión libre    0.009055
            Antiguedad_new-new    0.009000
                   Tipo_Plan_a    0.008944
Departamento_Santafé de Bogotá    0.008920

Importancias guardadas en: ../models/feature_importances.csv


# Conclusiones del Modelamiento Predictivo

## Resumen Ejecutivo

Se desarrolló un modelo de clasificación supervisada para predecir el comportamiento de clientes en el sistema de atención, evaluando 4 algoritmos diferentes sobre un dataset de 1,140,532 registros. El modelo seleccionado fue **Random Forest** con un ROC-AUC de **0.7420**.

## Comparación de Modelos

| Modelo | Accuracy | Precision | Recall | F1-Score | ROC-AUC | Tiempo (s) |
|--------|----------|-----------|--------|----------|---------|------------|
| **Random Forest** | **0.6717** | **0.4065** | **0.6720** | **0.5066** | **0.7420** | 48.22 |
| XGBoost | 0.5619 | 0.3112 | 0.6159 | 0.4135 | 0.6222 | 3.30 |
| LightGBM | 0.5502 | 0.3030 | 0.6103 | 0.4049 | 0.6065 | 3.17 |
| Logistic Regression | 0.4944 | 0.2572 | 0.5383 | 0.3480 | 0.5137 | 349.05 |

### Criterio de Selección

Random Forest fue seleccionado como el mejor modelo considerando:
- **ROC-AUC más alto**: 0.7420 (12% superior al segundo mejor)
- **Balance adecuado** entre precisión y recall
- **Tiempo de entrenamiento aceptable**: 48.22s (142x más rápido que Logistic Regression)

## Desempeño del Modelo Seleccionado

### Métricas Generales
- **Accuracy**: 67.17%
- **ROC-AUC**: 0.7420
- **F1-Score (Clase 1)**: 0.5066

### Métricas por Clase

| Métrica | Clase 0 (Negativa) | Clase 1 (Positiva) |
|---------|--------------------|--------------------|
| Precision | 0.86 | 0.41 |
| Recall | 0.67 | 0.67 |
| F1-Score | 0.75 | 0.51 |
| Soporte | 170,911 | 57,196 |

### Matriz de Confusión

|  | Predicho 0 | Predicho 1 |
|--|-----------|-----------|
| **Real 0** | 114,793 (TN) | 56,118 (FP) |
| **Real 1** | 18,761 (FN) | 38,435 (TP) |

**Interpretación:**
- **Verdaderos Negativos (TN)**: 114,793 - El modelo identifica correctamente el 67% de la clase negativa
- **Verdaderos Positivos (TP)**: 38,435 - El modelo captura el 67% de la clase positiva
- **Falsos Positivos (FP)**: 56,118 - 33% de la clase negativa es mal clasificada
- **Falsos Negativos (FN)**: 18,761 - 33% de la clase positiva no es detectada

## Variables Más Importantes

El análisis de feature importance revela los factores más influyentes en la predicción:

### Top 5 Variables
1. **Tiempo_en_espera** (9.10%) - Variable operativa crítica
2. **Ratio_Deuda_Edad** (8.83%) - Indicador financiero relativo
3. **Monto_adeudado** (8.79%) - Factor financiero directo
4. **Ratio_Duracion_Espera** (8.68%) - Eficiencia del servicio
5. **Duracion_llamada** (8.58%) - Complejidad de la consulta

### Variables Operativas (43%)
- Tiempo_en_espera
- Ratio_Duracion_Espera
- Duracion_llamada

### Variables Financieras (18%)
- Ratio_Deuda_Edad
- Monto_adeudado

### Variables Demográficas y Comportamentales (39%)
- Edad
- Uso de app
- Antigüedad del cliente
- Forma de pago
- Estado civil
- Recomendación de marca
- Historial de mora

## Hallazgos Clave

1. **Variables temporales dominan el modelo**: Las 3 variables más importantes están relacionadas con tiempos de espera y duración de llamadas, representando el 26% de la importancia total.

2. **Indicadores financieros relativos son más predictivos**: El ratio Deuda/Edad (8.83%) tiene mayor peso que variables absolutas, sugiriendo que el contexto del cliente es relevante.

3. **Desbalance de clases manejado efectivamente**: A pesar de la proporción 75/25, el modelo mantiene recall balanceado (67% en ambas clases).

4. **Trade-off precision-recall**: Alta precisión en clase negativa (86%) vs. baja en clase positiva (41%), reflejando el costo diferencial de errores tipo I y II.

## Limitaciones

1. **Precisión en clase minoritaria**: Solo 41% de precisión en la clase 1 implica alto número de falsos positivos (56,118).

2. **Interpretabilidad vs. Desempeño**: Random Forest ofrece mejor desempeño pero menor interpretabilidad que Logistic Regression.

3. **Tiempo de entrenamiento**: 48s puede ser limitante para reentrenamiento frecuente en producción.

## Recomendaciones

### Mejoras al Modelo
1. **Ajuste de threshold**: Evaluar umbrales de decisión diferentes a 0.5 para optimizar precision-recall según el costo de negocio de FP vs FN.

2. **Feature engineering adicional**: 
   - Interacciones entre variables temporales y financieras
   - Agregaciones por segmento de cliente
   - Variables lag de comportamiento histórico

3. **Técnicas de balanceo**: Evaluar SMOTE o undersampling para mejorar precision en clase minoritaria.

4. **Ensemble avanzado**: Combinar Random Forest con XGBoost/LightGBM para capturar patrones complementarios.

### Implementación en Producción
1. **Monitoreo continuo**: Tracking de ROC-AUC, precision y recall en datos nuevos para detectar drift.

2. **Reentrenamiento periódico**: Calendario mensual o basado en degradación de métricas.

3. **A/B Testing**: Validar impacto del modelo en KPIs de negocio (reducción de tiempo de atención, satisfacción del cliente).

4. **Explicabilidad**: Implementar SHAP values para explicar predicciones individuales a equipos de negocio.

## Conclusión

El modelo Random Forest desarrollado logra un desempeño sólido (ROC-AUC 0.74) en la predicción del comportamiento de clientes, con variables operativas de tiempo de espera como principales drivers. El modelo está listo para implementación en producción con las consideraciones mencionadas, y se espera que permita personalizar la atención según el perfil de riesgo predicho, optimizando recursos y mejorando la experiencia del cliente.